In [30]:
# Set up infrastructure and basic problem parameters
import multiprocessing as mp
import numpy as np
import datetime, os
from ContNoRegret.Domains import nBox, UnionOfDisjointnBoxes, DifferenceOfnBoxes, unitbox, hollowbox
from ContNoRegret.LossFunctions import random_PolynomialLosses, random_AffineLosses, random_QuadraticLosses
from ContNoRegret.NoRegretAlgos import ContNoRegretProblem
from ContNoRegret.utils import CNR_worker, plot_results, save_results, circular_tour
from ContNoRegret.animate import save_animations
from ContNoRegret.Potentials import (ExponentialPotential, IdentityPotential, pNormPotential, CompositePotential,
                                        ExpPPotential, PExpPotential, HuberPotential, LogtasticPotential, FractionalLinearPotential)

tmpfolder = '/Volumes/tmp/'

T = 100 # Time horizon
M = 10.0 # Uniform bound on the function (in the dual norm)
L = 5.0 # Uniform bound on the Lipschitz constant
N = 2500 # Number of parallel algorithm instances
Ngrid = 250000 # Number of gridpoints for the sampling step
H = 0.1 # strict convexity parameter (lower bound on evals of Q)

inf_to_2 = {'affine':[], 'quad_cvx':[], 'quad_noncvx':[], 'polynomial':[]}
inf_to_1 = {'affine':[], 'quad_cvx':[], 'quad_noncvx':[], 'polynomial':[]}

for n in np.arange(2,5):
    dom = unitbox(n)
    
    lossfuncs = {}
    
    # Now create some random loss functions
    lossfuncs['affine'], M = random_AffineLosses(dom, L, T)
    mus = dom.sample_uniform(T)
    lossfuncs['quad_cvx'], M, lambdamax = random_QuadraticLosses(dom, mus, L, M, pd=True)
    lossfuncs['quad_noncvx'], M, lambdamax = random_QuadraticLosses(dom, mus, L, M, pd=False)
    lossfuncs['polynomial'] = random_PolynomialLosses(dom, T, M, L, 4, [0,1,2,3,4])
    
    for key in inf_to_2.keys():
        # compute bounds on the norms
        normbounds = {'{}'.format(p): [lossfunc.norm(p, tmpfolder=tmpfolder) for lossfunc in lossfuncs[key]] for p in [1,2,np.Infinity]}
        # compute ratios
        inf_to_2[key].append(np.max(np.array(normbounds['inf'])/np.array(normbounds['2'])))
        inf_to_1[key].append(np.max(np.array(normbounds['inf'])/np.array(normbounds['1'])))

In [28]:
np.min(inf_to_2), np.average(inf_to_2), np.max(inf_to_2)

(3.4993136328385912, 3.7260293786881715, 3.7945399746381052)

In [29]:
np.min(inf_to_1), np.average(inf_to_1), np.max(inf_to_1)

(1.389677913537926, 2.9094135182403851, 3.2154490708251329)

In [31]:
inf_to_2

{'affine': [2.7774601434023172, 3.7930646876158076, 4.8015310073783208],
 'polynomial': [6.7082039325186011, 5.1912906903564489, 14.600751249963142],
 'quad_cvx': [4.2771686762193806, 5.2558429935222337, 7.2321817682353382],
 'quad_noncvx': [3.9071864388602418, 4.216804757199232, 4.0604697287771314]}

In [32]:
inf_to_1


{'affine': [2.3296387405891101, 3.2342992216369821, 4.1435496368932059],
 'polynomial': [15.00000000016758, 7.2882997122074391, 35.510747526635761],
 'quad_cvx': [6.4909152996477602, 7.4037673439247964, 11.089352055707234],
 'quad_noncvx': [5.7934698707404912, 5.6729607648143308, 4.5787443930601608]}

In [33]:
import matplotlib.pyplot as plt

In [34]:
plt.plot(inf_to_2)

TypeError: float() argument must be a string or a number, not 'dict'